

> # **USE AUTOMATIC1111 VIA GOOGLE COLAB**





In [ ]:
#@title Keep Colab Alive
# This Code Prevents Google Colab from Timing out due to Inactivity.
import torch
import time
import IPython
from IPython.display import display, HTML
import threading

# Function to keep Colab connected
def keep_alive():
    while True:
        # Execute a small GPU operation to keep the session active
        if torch.cuda.is_available():
            # Small tensor operation on GPU to keep it active
            torch.cuda.synchronize()
            x = torch.randn(100, 100, device='cuda')
            y = x + x
            del x, y
            torch.cuda.empty_cache()

        # Sleep for 45 seconds - Colab disconnects after ~90 seconds of inactivity
        time.sleep(45)

# Start the keep-alive function in a background thread
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

In [ ]:
#@title Installing Miniconda with Python 3.10.6

# Conda Env Name : py310
# Location : /opt/conda/bin/conda
# Activate Env : !source /opt/conda/bin/activate py310

import os
import subprocess

conda_dir = "/opt/conda"
conda_bin = os.path.join(conda_dir, "bin", "conda")
env_name = "py310"
python_version = "3.10.6"

def run(cmd, shell=False):
    if shell:
        subprocess.run(cmd, shell=True, check=True)
    else:
        subprocess.run(cmd.split(), check=True)

def install_miniconda():
    run(f"curl -o Miniconda3.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_23.3.1-0-Linux-x86_64.sh", shell=True)
    run("chmod +x Miniconda3.sh")
    run(f"bash ./Miniconda3.sh -b -f -p {conda_dir}", shell=True)
    os.remove("Miniconda3.sh")

def create_conda_env():
    run(f"{conda_bin} create -y -n {env_name} python={python_version}")

if not os.path.exists(conda_bin):
    install_miniconda()

create_conda_env()


In [ ]:
#@title Clone Automatic1111 & Install Popular Extension

import os
import subprocess

# Config
conda_setup = "source /opt/conda/etc/profile.d/conda.sh"
env_name = "py310"
base_dir = "/content"
webui_dir = os.path.join(base_dir, "stable-diffusion-webui")
extensions = [
    "https://github.com/Mikubill/sd-webui-controlnet",
    "https://github.com/deforum-art/deforum-for-automatic1111-webui",
    "https://github.com/camenduru/stable-diffusion-webui-images-browser",
    "https://github.com/Bing-su/adetailer.git",
    "https://github.com/hako-mikan/sd-webui-regional-prompter",
    "https://github.com/AUTOMATIC1111/stable-diffusion-webui-promptgen",
    "https://github.com/BlafKing/sd-civitai-browser-plus",
    "https://github.com/dan4in/PonyHelper"
]

# Clone Automatic1111 if not present
if not os.path.exists(webui_dir):
    subprocess.run(
        f"git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui {webui_dir}",
        shell=True, executable="/bin/bash"
    )

# Clone each extension into the extensions folder
for ext in extensions:
    ext_name = ext.rstrip("/").split("/")[-1]
    ext_path = os.path.join(webui_dir, "extensions", ext_name)
    if not os.path.exists(ext_path):
        subprocess.run(
            f"{conda_setup} && conda activate {env_name} && cd {webui_dir}/extensions && git clone {ext}",
            shell=True,
            executable="/bin/bash"
        )

In [ ]:
#@title CivitAi Model Downloader Via API
import os
import re
import sys
import requests
from tqdm import tqdm

# Set your CivitAI API key here
API_KEY = "API_KEY"  # ← set your CivitAI key

# ─── Paste your URLs here ────────────────────────────────────────────────
# Base directory paths
CKPT_PATH = "/content/stable-diffusion-webui/models/Stable-diffusion"
LORA_PATH = "/content/stable-diffusion-webui/models/Lora"
VAE_PATH = "/content/stable-diffusion-webui/models/VAE"
CONTROLNET_PATH = "/content/stable-diffusion-webui/models/ControlNet"
EMBEDDING_PATH = "/content/stable-diffusion-webui/embeddings"

ckpt_links = [
    "https://civitai.com/models/372465?modelVersionId=914390",
    "https://civitai.com/models/439889?modelVersionId=1199750"
]

lora_links = [
    "https://civitai.com/models/696418?modelVersionId=779327",
    "https://civitai.com/models/1327644?modelVersionId=1498957",
    "https://civitai.com/models/1333749/add-detail-slider",
    "https://civitai.com/models/669571/pony-add-more-details",
    "https://civitai.com/models/454172?modelVersionId=505635",
    "https://civitai.com/models/300005?modelVersionId=436219",
    "https://civitai.com/models/73756?modelVersionId=703107"
]

vae_links = [
    "https://civitai.com/models/199948?modelVersionId=224985"
]

embedding_links = [
    "https://civitai.com/models/118418?modelVersionId=134583",
    "https://civitai.com/models/583583?modelVersionId=651084"
]

controlnet_links = [
    # add ControlNet URLs here...
]

# ──────────────────────────────────────────────────────────────────────────
# Check if API key is set
if not API_KEY:
    print("API key not set. Please set your CivitAI API key.")
    sys.exit(1)

# Create all required directories
os.makedirs(CKPT_PATH, exist_ok=True)
os.makedirs(LORA_PATH, exist_ok=True)
os.makedirs(VAE_PATH, exist_ok=True)
os.makedirs(CONTROLNET_PATH, exist_ok=True)
os.makedirs(EMBEDDING_PATH, exist_ok=True)

def parse_mid_and_override(url):
    m = re.search(r"models/(\d+)", url)
    if not m:
        raise ValueError(f"Bad URL: {url}")
    mid = m.group(1)
    over = re.search(r"modelVersionId=(\d+)", url)
    return mid, (over.group(1) if over else None)

def get_vid(mid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/models/{mid}", headers=h, timeout=10
    )
    r.raise_for_status()
    return r.json()["modelVersions"][0]["id"]

def fetch_dl_info(vid):
    h = {"Authorization": f"Bearer {API_KEY}"}
    r = requests.get(
        f"https://civitai.com/api/v1/model-versions/{vid}", headers=h, timeout=10
    )
    r.raise_for_status()
    j = r.json()
    return j["downloadUrl"], j["files"][0]["name"]

def download(url, dest_folder):
    try:
        mid, override = parse_mid_and_override(url)
        vid = override or get_vid(mid)
        dl_url, fname = fetch_dl_info(vid)
        print(f"Processing: {url}")
        print(f"Model ID: {mid}, Version ID: {vid}")
        print(f"Filename: {fname}")
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return

    path = os.path.join(dest_folder, fname)

    if os.path.exists(path):
        print(f"File already exists: {path}")
        return

    print(f"Downloading to: {path}")

    try:
        r = requests.get(
            dl_url,
            headers={"Authorization": f"Bearer {API_KEY}"},
            stream=True
        )

        if r.status_code != 200:
            print(f"Download failed with status code {r.status_code}")
            return

        total = int(r.headers.get("content-length", 0))
        bar = tqdm(total=total, unit="iB", unit_scale=True, desc=fname)

        with open(path, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
                bar.update(len(chunk))

        bar.close()
        print(f"Downloaded: {fname}")
    except Exception as e:
        print(f"Error downloading {fname}: {e}")
        if os.path.exists(path):
            os.remove(path)  # Remove partially downloaded file

# ─── Kick off downloads ───────────────────────────────────────────────────
print("Starting downloads...")

print("\nDownloading checkpoint models...")
for u in ckpt_links:
    download(u, CKPT_PATH)

print("\nDownloading LoRA models...")
for u in lora_links:
    download(u, LORA_PATH)

print("\nDownloading VAE models...")
for u in vae_links:
    download(u, VAE_PATH)

print("\nDownloading embedding models...")
for u in embedding_links:
    download(u, EMBEDDING_PATH)

print("\nDownloading ControlNet models...")
for u in controlnet_links:
    download(u, CONTROLNET_PATH)

print("\n✨ All downloads completed.")

In [ ]:
#@title Activate Environment and Install Multiple (Extra) Packages
!source /opt/conda/bin/activate py310 && pip install insightface

# Example with different packages
#source /opt/conda/bin/activate py310 && pip install pandas requests scikit-learn

In [ ]:
#@title Launch WebUI : Might Take 10/15 Mins
%env MPLBACKEND=agg
!source /opt/conda/bin/activate py310 && python /content/stable-diffusion-webui/launch.py --medvram --no-half-vae --xformers --share